# Tool을 사용하는 챗봇 구현

## OpenAI LLM 준비
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`

In [ ]:
import gradio as gr
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain.tools import tool


# .env 파일 로드 및 API 키 확인
load_dotenv()

# OpenAI LLM 준비
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

In [ ]:
## 단계 - 도구 구성
@tool
def tranquilizer_watch(target: str) -> str:
    """마취 시계: 지정된 대상을 잠재울 필요가 있을 때 사용합니다. 추리 설명 등을 대신할 때 유용합니다.
    Args:
        target (str): 마취시킬 대상의 이름이나 인상착의. 예: '안경 쓴 범인', '유명한 탐정님'
    """
    return f"⌚ 마취 시계: '{target}'을(를) 성공적으로 마취시켰습니다."


@tool
def voice_changer_bowtie(target: str) -> str:
    """음성 변조 나비넥타이: 다른 사람의 목소리로 추리를 설명하거나, 다른 사람인 척 연기해야 할 때 사용합니다.
    Args:
        target (str): 목소리를 흉내 낼 대상. 예: '브라운 박사님', '유명한 탐정님'
    """
    return f"🎤 음성 변조 나비넥타이: '{target}'의 목소리로 변조를 시작합니다."


@tool
def detective_glasses(target: str) -> str:
    """탐정 안경: 특정 대상을 추적하거나 멀리 있는 것을 확대해서 볼 때 사용합니다. 범인 추적에 필수적입니다.
    Args:
        target (str): 추적하거나 확대할 대상. 예: '범인의 자동차', '먼 곳의 단서'
    """
    return f"🕶️ 탐정 안경: '{target}'에 대한 추적 및 확대 기능을 활성화합니다."


@tool
def soccer_shoes(target: str) -> str:
    """킥력 강화 축구화: 강력한 힘으로 무언가를 걷어차 범인을 제압하거나 위기 상황을 탈출할 때 사용합니다.
    Args:
        target (str): 강하게 찰 대상. 예: '범인을 위협할 돌멩이', '막다른 길의 문'
    """
    return f"⚽ 킥력 강화 축구화: '{target}'을(를) 향해 강력한 킥을 준비합니다!"

In [ ]:
# 도구 목록 정리
tools = [tranquilizer_watch, voice_changer_bowtie, detective_glasses, soccer_shoes]

# 도구 목록을 LLM에 연결
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage, ToolMessage

class AgentState(TypedDict, total=False):
    messages: Annotated[list, "LLM 메시지 목록", add_messages]

SYSTEM_PROMPT = """당신은 명탐정 코난입니다. 주어진 상황을 해결하기 위해 당신이 가진 도구들을 적절하게 사용하세요. 상황에 따라 여러 도구를 동시에 사용할 수도 있습니다."""
def ensure_system_once(msgs: list[BaseMessage]) -> list[BaseMessage]:
    """SystemMessage가 계속 누적되는 것을 방지"""
    if not msgs or not isinstance(msgs[0], SystemMessage):
        return [SystemMessage(content=SYSTEM_PROMPT)] + msgs
    return msgs

def agent_node(state: AgentState) -> AgentState:
    """LLM에 상태의 메시지들을 전달하고 응답을 받아 상태에 추가"""
    messages = ensure_system_once(state.get("messages", []))
    ai = llm_with_tools.invoke(messages)
    return {"messages": [ai]}


In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

graph = StateGraph(AgentState)

graph.add_node("agent", agent_node)
graph.add_node("tools", ToolNode(tools))

graph.add_edge(START, "agent")
graph.add_conditional_edges(
    "agent",
    tools_condition,
    {
        "tools": "tools",   # tool_calls 있으면 ToolNode로
        "__end__": END,     # 없으면 종료
    },
)


app = graph.compile()

## Gradio UI 구성
### Gradio 처리 함수 설정

In [ ]:
def play_chat(user_input: str) -> str:
    
    if not user_input:
        return "상황을 입력해주세요."

    result_state =  app.invoke({"messages": [HumanMessage(content=user_input)]})
    messages = result_state["messages"]
    output = []
    for i, msg in enumerate(messages):
        if isinstance(msg, ToolMessage):
            output.append(f"(도구 사용) {msg.content}")
    return "\n".join(output)

### Gradio UI 구성

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("### 🕵️ 명탐정 코난 도구 추천기")
    gr.Markdown("상황을 입력하면 코난이 상황에 적절한 도구를 사용하게 됩니다.")

    user_input = gr.Textbox(label="상황 설명", placeholder="예: 모리 탐정을 기절시키고, 모리 탐정 목소리로 사건을 설명하고 싶어요")
    ai_output = gr.Textbox(label="추천 도구", lines=5)
    user_input.submit(play_chat, inputs=user_input, outputs=ai_output)

demo.launch()

In [ ]:
demo.close()

-----
** End of Documents **